In [1]:
import os
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix

In [2]:
val_submission = pd.read_csv('../data/val_submission/bilstm_cnn_withfasttext.csv')

val = pd.read_csv('../data/subtask1_dev.csv')
val = val[val.tag != 'NULL']
val = val[pd.notnull(val.tag)]
print (val.shape, val_submission.shape)

(49051, 7) (49051, 9)


In [3]:
train = pd.read_csv('../data/subtask1_train.csv')[['word','tag']]
val = pd.read_csv('../data/subtask1_dev.csv')[['word','tag']]
all_words = pd.concat([train,train],axis=0).drop_duplicates().reset_index(drop=True)

xx = all_words.groupby(['word'])['tag'].nunique().reset_index()
xx.columns = ['word','tag_counts']
xx = xx[xx.tag_counts == 1]
print (xx.shape)

uniq_word_tags = pd.merge(all_words[all_words.tag != 'OTHER'],xx,how='inner')[['word','tag']]
uniq_word_tags.columns = ['word','actual_tag']
uniq_word_tags = uniq_word_tags.drop_duplicates(subset=['word'])

val_submission = pd.merge(val_submission,uniq_word_tags,how='left')

def fix_tag(x):
    if pd.notnull(x.actual_tag) and x.tag_pred != x.actual_tag:
        x['tag_pred'] = x.actual_tag
    return x

val_submission = val_submission.apply(lambda x: fix_tag(x),axis=1)

(19350, 2)


In [4]:
val = val.reset_index(drop=True)

In [5]:
val_submission.head(10)

,document,sentence,n1,n2,word,tag,pos,doc_sent,tag_pred,actual_tag
0,S1139-76322016000300016-2,0,0,8,lactante,OTHER,ADV,S1139-76322016000300016-2_0,OTHER,NaN
1,S1139-76322016000300016-2,0,12,13,1,OTHER,NUM,S1139-76322016000300016-2_0,OTHER,NaN
2,S1139-76322016000300016-2,0,14,17,mes,OTHER,NOUN,S1139-76322016000300016-2_0,OTHER,NaN
3,S1139-76322016000300016-2,0,20,22,29,OTHER,NUM,S1139-76322016000300016-2_0,OTHER,NaN
4,S1139-76322016000300016-2,0,23,27,días,OTHER,NOUN,S1139-76322016000300016-2_0,OTHER,NaN
5,S1139-76322016000300016-2,0,33,45,antecedentes,OTHER,NOUN,S1139-76322016000300016-2_0,OTHER,NaN
6,S1139-76322016000300016-2,0,49,56,interés,OTHER,NOUN,S1139-76322016000300016-2_0,OTHER,NaN
7,S1139-76322016000300016-2,0,62,67,acude,OTHER,VERB,S1139-76322016000300016-2_0,OTHER,NaN
8,S1139-76322016000300016-2,0,70,79,urgencias,OTHER,PROPN,S1139-76322016000300016-2_0,OTHER,NaN
9,S1139-76322016000300016-2,0,83,92,pediatría,OTHER,VERB,S1139-76322016000300016-2_0,OTHER,NaN


In [6]:
val_submission = val_submission[val_submission.tag != 'OTHER'][val_submission.tag != 'UNCLEAR']

/Users/victor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [7]:
val_submission[val_submission.tag != val_submission.tag_pred].head(10)

,document,sentence,n1,n2,word,tag,pos,doc_sent,tag_pred,actual_tag
619,S0004-06142009000400011-1,5,2216,2231,citoquerqatinas,PROTEINAS,NOUN,S0004-06142009000400011-1_5,OTHER,NaN
947,S1130-14732009000300006-1,0,2478,2490,homocisteina,NORMALIZABLES,NOUN,S1130-14732009000300006-1_0,OTHER,NaN
1247,S0212-16112005000600011-1,1,2224,2236,liposolubles,NORMALIZABLES,ADJ,S0212-16112005000600011-1_1,OTHER,NaN
1249,S0212-16112005000600011-1,1,2255,2256,a,NORMALIZABLES,ADP,S0212-16112005000600011-1_1,OTHER,NaN
1269,S0212-16112005000600011-1,1,2428,2436,complejo,NORMALIZABLES,ADJ,S0212-16112005000600011-1_1,OTHER,NaN
1270,S0212-16112005000600011-1,1,2437,2438,b,NORMALIZABLES,NOUN,S0212-16112005000600011-1_1,OTHER,NaN
1331,S0212-16112005000600011-1,2,3009,3015,auxina,NORMALIZABLES,ADJ,S0212-16112005000600011-1_2,OTHER,NaN
1332,S0212-16112005000600011-1,2,3016,3019,a+e,NORMALIZABLES,PROPN,S0212-16112005000600011-1_2,OTHER,NaN
1565,S0210-56912007000700008-1,1,555,576,lacticodeshidrogenasa,PROTEINAS,NOUN,S0210-56912007000700008-1_1,OTHER,NaN
1660,S0210-56912007000700008-1,2,1620,1634,succinilcolina,NORMALIZABLES,ADJ,S0210-56912007000700008-1_2,OTHER,NaN


In [8]:
val_submission[pd.notnull(val_submission.tag_pred) == False]

,document,sentence,n1,n2,word,tag,pos,doc_sent,tag_pred,actual_tag


In [9]:
print (val_submission.shape)
print (val_submission.tag.nunique())

(1897, 10)
3


In [10]:
val_submission[val_submission.document == 'S0004-06142006000300015-1']

,document,sentence,n1,n2,word,tag,pos,doc_sent,tag_pred,actual_tag
16893,S0004-06142006000300015-1,0,1711,1714,ema,PROTEINAS,PROPN,S0004-06142006000300015-1_0,PROTEINAS,PROTEINAS
16894,S0004-06142006000300015-1,0,1716,1718,ck,PROTEINAS,NOUN,S0004-06142006000300015-1_0,PROTEINAS,PROTEINAS
16895,S0004-06142006000300015-1,0,1719,1720,7,PROTEINAS,NUM,S0004-06142006000300015-1_0,OTHER,NaN
16897,S0004-06142006000300015-1,0,1733,1737,ck20,PROTEINAS,ADJ,S0004-06142006000300015-1_0,PROTEINAS,PROTEINAS


In [11]:
print (accuracy_score(val_submission.tag,val_submission.tag_pred))
print (f1_score(val_submission.tag,val_submission.tag_pred,average='macro'))

0.7496046389035319
0.3919777839409881


/Users/victor/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [12]:
valy_actual = pd.DataFrame(columns = ['document','n1','n2','tag'])
valy_pred = pd.DataFrame(columns = ['document','n1','n2','tag_pred'])

In [13]:
i = 0
j = 0

n1 = val_submission['n1'].iloc[i]
n2 = val_submission['n2'].iloc[i]
document = val_submission['document'].iloc[i]
doc_sent = val_submission['doc_sent'].iloc[i]
tag = val_submission['tag'].iloc[i]
word = [val_submission['word'].iloc[i]]

while i < val_submission.shape[0]-1:

    if val_submission['doc_sent'].iloc[i+1] == doc_sent and \
            val_submission['tag'].iloc[i+1] == tag and val_submission['n1'].iloc[i+1] == n2 + 1 :
            
            n2 = val_submission['n2'].iloc[i+1]
            word += [val_submission['word'].iloc[i+1]]
    
    else:
        
        valy_actual.loc[j,'document'] = document
        valy_actual.loc[j,'n1'] = n1
        valy_actual.loc[j,'n2'] = n2
        valy_actual.loc[j,'tag'] = tag
        valy_actual.loc[j,'words'] = " ".join(word)
        j += 1
        
        n1 = val_submission['n1'].iloc[i+1]
        n2 = val_submission['n2'].iloc[i+1]
        document = val_submission['document'].iloc[i+1]
        doc_sent = val_submission['doc_sent'].iloc[i+1]
        tag = val_submission['tag'].iloc[i+1]
        word = [val_submission['word'].iloc[i+1]]

    i += 1

In [14]:
i = 0
j = 0

n1 = val_submission['n1'].iloc[i]
n2 = val_submission['n2'].iloc[i]
document = val_submission['document'].iloc[i]
doc_sent = val_submission['doc_sent'].iloc[i]
tag = val_submission['tag_pred'].iloc[i]
word = [val_submission['word'].iloc[i]]

while i < val_submission.shape[0]-1:
     
    if val_submission['doc_sent'].iloc[i+1] == doc_sent and \
            val_submission['tag_pred'].iloc[i+1] == tag and val_submission['n1'].iloc[i+1] == n2 + 1 :
            
            n2 = val_submission['n2'].iloc[i+1]
            word += [val_submission['word'].iloc[i+1]]
    
    else:
        
        valy_pred.loc[j,'document'] = document
        valy_pred.loc[j,'n1'] = n1
        valy_pred.loc[j,'n2'] = n2
        valy_pred.loc[j,'tag_pred'] = tag
        valy_pred.loc[j,'words_pred'] = " ".join(word)
        
        j += 1
        
        n1 = val_submission['n1'].iloc[i+1]
        n2 = val_submission['n2'].iloc[i+1]
        document = val_submission['document'].iloc[i+1]
        doc_sent = val_submission['doc_sent'].iloc[i+1]
        tag = val_submission['tag_pred'].iloc[i+1]
        word = [val_submission['word'].iloc[i+1]]

    i += 1

In [15]:
valy_pred = valy_pred[valy_pred.tag_pred != 'OTHER'][valy_pred.tag_pred != 'PAD']

/Users/victor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [16]:
valy = pd.merge(valy_actual,valy_pred,how='outer')
valy['tag'] = valy['tag'].fillna('DUMMY')
valy['tag_pred'] = valy['tag_pred'].fillna('DUMMY')

In [17]:
print (accuracy_score(valy.tag,valy.tag_pred))
print (f1_score(valy.tag,valy.tag_pred,average='macro'))

0.6901241868716735
0.4904165695330421


In [18]:
confusion_matrix(valy.tag,valy.tag_pred)

array([[  0,  33,   1,  58],
       [253, 687,   0,   7],
       [  6,   1,   2,   0],
       [162,   3,   0, 478]])

In [19]:
valy[valy.tag != valy.tag_pred].head(10)

,document,n1,n2,tag,words,tag_pred,words_pred
9,S0004-06142009000400011-1,2216.0,2231.0,PROTEINAS,citoquerqatinas,DUMMY,NaN
14,S1130-14732009000300006-1,2478.0,2490.0,NORMALIZABLES,homocisteina,DUMMY,NaN
22,S0212-16112005000600011-1,2214.0,2236.0,NORMALIZABLES,vitaminas liposolubles,DUMMY,NaN
23,S0212-16112005000600011-1,2255.0,2256.0,NORMALIZABLES,a,DUMMY,NaN
25,S0212-16112005000600011-1,2428.0,2438.0,NORMALIZABLES,complejo b,DUMMY,NaN
29,S0212-16112005000600011-1,3009.0,3019.0,NORMALIZABLES,auxina a+e,DUMMY,NaN
36,S0210-56912007000700008-1,555.0,576.0,PROTEINAS,lacticodeshidrogenasa,DUMMY,NaN
39,S0210-56912007000700008-1,1620.0,1634.0,NORMALIZABLES,succinilcolina,DUMMY,NaN
43,S0210-56912007000700008-1,3260.0,3275.0,NORMALIZABLES,solución salina,DUMMY,NaN
44,S0210-56912007000700008-1,3501.0,3505.0,NORMALIZABLES,fio2,DUMMY,NaN
